# Tutorial 05: Homework

### Assignement
  * For the attached data (cleaned data from the 3rd tutorial) try to use linear regression (or ridge regression, if you will) AND decision tree regressor (or boosted tree) applied on features given by PCA.
  * Try to select as less as possible features so that the results are comparably good (in terms of RMSLE) to the results obtained in the 3rd tutorial (RMSLE around 0.12).
  * Experiment with the feature selection:
    * Has removing indicator or discrete features a positive influence?
    * Are the first principal components always the best choice?

# Import libraries

In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import AdaBoostRegressor
import pandas as pd
import numpy as np
import math
from scipy import stats
import math
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import matplotlib
from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import model_selection, linear_model, metrics, preprocessing, feature_selection
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, recall_score
%matplotlib inline
matplotlib.style.use('ggplot')

# Load and split data

In [ ]:
# loading data
data = pd.read_csv('dataHW05.csv', index_col=0)
# spliting data to train and testdatasets
dtrain, dtest = train_test_split(data, test_size=0.25, random_state=17) #there was 458

# Select columns with very low variance to delete

In [ ]:
treshold = 0.001
# Id neither help with predictions
columns_to_remove = ['Id']
columns_to_remove = list(set(list(dtrain.columns[dtrain.var() < treshold]) + columns_to_remove))

#print number of columns to delete
display(len(columns_to_remove))

# Prepare functions to check linear regression without some columns

In [ ]:
def check_without_columns(train,validate,columns):
    tmp_t = train.drop(columns,axis = 1, errors = 'ignore')
    tmp_v = validate.drop(columns,axis = 1, errors = 'ignore')
    linreg(tmp_t,tmp_v)
    print()
    
def linreg(train, validate):
    # Data prepare
    X = train.drop(['SalePrice'], axis = 1, errors = 'ignore')
    y = train.SalePrice
    Xv = validate.drop(['SalePrice'], axis = 1, errors = 'ignore')
    yv = validate.SalePrice   
    
    # Linear Regression train
    clf = LinearRegression()
    clf.fit(X, y) 
    
    # Print RMSLE
    print('Linear regression root mean squared validation error:',
          np.sqrt(mean_squared_error(clf.predict(Xv), yv)))

Check this result

In [ ]:
check_without_columns(dtrain,dtest,columns_to_remove)

# Add columns, which are not correlated with target value, to delete

In [ ]:
corrS = dtrain.drop(columns_to_remove, axis = 1, errors = 'ignore').corr(method='spearman')
print(corrS.shape)
treshold = 0.01
col_S = list(corrS.SalePrice[corrS.SalePrice.abs() < treshold].index)
columns_to_remove_S = list(set(columns_to_remove + col_S))
print("After Spearman",len(columns_to_remove_S))

# Check T-test for indicator variables and choose some to delete 

In [ ]:
ttest_pvals = data\
    .drop(columns_to_remove, axis = 1, errors = 'ignore')\
    .select_dtypes(include = ['int64']).columns\
    .to_series()\
    .apply(lambda x: stats.ttest_ind(data.SalePrice[data[x] == 0], data.SalePrice[data[x] == 1], equal_var = False).pvalue)

val = 0.5
columns_to_remove_tt = list(set(columns_to_remove_S + list(ttest_pvals[ttest_pvals > val].index)))

print(len(columns_to_remove_tt))


# Prepare function to apply wrapper method
I have choosen rfecv, because I got better results than with Lasso method

In [ ]:
def wrapper_rfecv(dtrain,dtest):
    X = dtrain.drop(['SalePrice'], axis = 1, errors = 'ignore')
    y = dtrain.SalePrice

    used_columns = X.columns

    # define the scorer which will be the RMSE
    def scorer(Y, yp):
        return np.sqrt(mean_squared_error(Y, yp)) #tu bylo metrics.

    # prepare the model
    clfM = linear_model.LinearRegression()

    # prepare the backward selection algorithm (recursive feature elimination with cross validation)
    selector = feature_selection.RFECV(clfM, step=1, cv=4, scoring=metrics.make_scorer(scorer)) # there was 6
    # run it
    selector = selector.fit(X, y)

    # transform result to dataframe
    result = pd.DataFrame({'Chosen': selector.support_, 'Ranking': selector.ranking_}, index=list(used_columns))
    # columns to leave
    columns_to_leave = result[result.Chosen == True].index
    # show results
    print('Chosen', len(columns_to_leave),'from ', result.Chosen.shape[0], 'features.')
    
    return dtrain[list(columns_to_leave) + ['SalePrice']],dtest[list(columns_to_leave) + ['SalePrice']]

### Apply wrapper method on the data set without columns which I decided to remove

In [ ]:
final_columns_to_remove = columns_to_remove_tt

dtrain = dtrain.drop(final_columns_to_remove, axis = 1, errors = 'ignore')
dtest = dtest.drop(final_columns_to_remove, axis = 1, errors = 'ignore')

#wrapper method return the final result of training and testing data set
dtrain_rfecv,dtest_rfecv = wrapper_rfecv(dtrain,dtest)

## Prepare function to check linear regression for final data set on which I applied PCA
I decided to use just linear regression, because I checked that applying additionally decision tree regressor (or boosted tree) didn't improve my results.

After wrapper algorithm there was 232 columns left so I decided to check how PCA will help for maximum 200 first principal columns chosen. I have chosen 200, because where I chosen bigger then results were higher for this few more attempts and it disturbed the plot.

In [ ]:
def lin_reg_with_pca(dtrain,dtest):
    q = 205 # number of components

    X = dtrain.drop(['SalePrice'], axis = 1, errors = 'ignore')
    y = dtrain.SalePrice
    Xtest = dtest.drop(['SalePrice'], axis = 1, errors = 'ignore')
    ytest = dtest.SalePrice

    ####
    # without pca
    ####
    # Linear Regression
    clf1 = LinearRegression()
    clf1.fit(X, y) 

    # Print RMSLE
    RMSLE_OLS = np.sqrt(mean_squared_error(clf1.predict(Xtest), ytest))
    print('Root mean squared logarithmic error:', RMSLE_OLS)
    
    
    #### 
    # pca with and without scaling
    ####
    pca = PCA()
    pca.fit_transform(X)
    X1 = pca.transform(X)
    Xtest1 = pca.transform(Xtest)

    pca.fit_transform(scale(X))
    X2 = pca.transform(scale(X))
    Xtest2 = pca.transform(scale(Xtest))
    RMSLE = []
    RMSLE_scale = []
    
    rng = np.random.RandomState(1)
    
    for n in range(1,q):
        Xsub1 = X1[:,0:n]
        Xsubtest1 = Xtest1[:,0:n]
        clf1 = LinearRegression()
        clf1.fit(Xsub1, y)
        predicted_from_clf1 = clf1.predict(Xsubtest1)
        
        # save RMSLE
        RMSLE.append(np.sqrt(mean_squared_error(predicted_from_clf1, ytest)))        
        
        Xsub2 = X2[:,0:n]
        Xsubtest2 = Xtest2[:,0:n]
        clf1 = LinearRegression()
        clf1.fit(Xsub2, y) 
        predicted_from_clf1 = clf1.predict(Xsubtest2)
        
        # save RMSLE
        RMSLE_scale.append(np.sqrt(mean_squared_error(predicted_from_clf1, ytest)))
        
    plt.subplots(1,1, figsize=(15, 8))
    ns = plt.scatter(range(1,q), RMSLE, c='red')
    s = plt.scatter(range(1,q), RMSLE_scale, c='green')
    plt.title(u"RMSLE as a function of number of principal components used")
    plt.xlabel(u'number of principal component used')
    plt.ylabel('RMSLE')
    plt.plot([0, q], [RMSLE_OLS, RMSLE_OLS],'b-')
    plt.legend((ns,s),('non-scaled', 'scaled'))

    print("Number of first principals from pca with minimal RMSLE for not scaled data: " + str(np.argmin(RMSLE)+1))
    print("This minimal value for not scaled data: " + str(RMSLE[np.argmin(RMSLE)]))
    
    print("\n")

    print("Number of first principals from pca with minimal RMSLE for scaled data: " + str(np.argmin(RMSLE_scale)+1))
    print("This minimal value for scaled data: " + str(RMSLE_scale[np.argmin(RMSLE_scale)]))
    
    print("\n")

# Finally I check result on the data after feature selection and PCA

In [ ]:
lin_reg_with_pca(dtrain_rfecv,dtest_rfecv)

I can see that choosing 166 of 232 features by PCA (without scaling the data) let us get the best result - around 0.1224. For feature selection I used methods which helped me to get the best score in the 3rd homework so it is the best result which I can achieve.

I can notice that when I have chosen just few first principals with PCA then the result was not good. For 40 first principals with PCA the result get closer to the value of RMSLE for data without PCA, and when we were choosing more and more components than there was an improvement with a result to the best value. But the best value was get not for the all components, so PCA works well.